# Manage Imports

In [1]:
import numpy as np
import gym
import random

# Initialize Environment

In [2]:
slippery = False # set to false for a deterministic game
env = None

if (slippery):
    env = gym.make("FrozenLake8x8-v0")
else:
    from gym.envs.registration import register

    register(
        id='FrozenLakeNotSlippery-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '8x8', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )

    env = gym.make("FrozenLakeNotSlippery-v0")

def has_won(state):
    return state == 63

# Create Q-Table

Create an $M \times N$ matrix, with 
- $M$ = size of action space
- $N$ = size of state space / observation space

In [3]:
action_space_size = env.action_space.n
observation_space_size = env.observation_space.n

print("Action Space: {}, Observation Space: {}".format(action_space_size, observation_space_size))

qtable = np.zeros((observation_space_size, action_space_size))
#print(qtable)

Action Space: 4, Observation Space: 64


# Setup Hyper Parameters

In [4]:
total_training_episodes = 100000
total_play_episodes = 10000
learning_rate = 0.8
max_steps = 9999

#discounting rate
gamma = 0.95

#exploration rate
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01

#exponential decay rate for exploration probability
decay_rate = 1 / total_training_episodes
print(decay_rate)

1e-05


# Output Setup

In [5]:
render = False
verbosity = 0 # 0=light, 1=medium, 2=heavy, 3=desperate housewifes
log_every_n_episode = 10000

# The Q-Learning Algorithm

In [6]:
total_training_reward = 0
successful_training_episodes = 0
epsilon = max_epsilon

print("Episode\t| Steps survived\t| Epsilon \t| Average Reward\t| Successful Episodes")

for episode in range(total_training_episodes):
    if (verbosity > 1):
        print("===================================")
        print("Starting Episode {}".format(episode))
        
    state = env.reset()
    reward = 0
    done = False
    info = {'prob': 0}

    total_reward = 0
    step = 0
    
    while (not done and step < max_steps):        
        # step 1: choose action (explore or exploit)
        explore = random.random() < epsilon

        # step 2: take action
        action = env.action_space.sample() if explore else np.argmax(qtable[state])
        new_state, reward, done, info = env.step(action)
        
        #if (custom_rewards):
        #    reward = rewards[new_state]
            
        total_reward += reward
        
        if (verbosity > 2):
            print("Step: {}, Epsilon: {}, Observation: {}, Reward: {}".format(new_state, epsilon, new_state, reward))
        
        if (render):
            env.render()
            
        # step 3: update q-table
        # Q'(s, a) := Q(s, a) + lr * (R(s, a) + gamma * max(Q(s', a')) - Q(s, a))
        qtable[state][action] = qtable[state][action] + learning_rate * (reward + gamma * max(qtable[new_state]) - qtable[state, action])
            
        if (verbosity > 2):
            print(qtable)
        
        state = new_state
        step += 1
    
    total_training_reward += total_reward
    
    if (has_won(state)):
        successful_training_episodes += 1
    
    if (verbosity > 1 or (episode + 1) % log_every_n_episode == 0 or (verbosity > 0 and has_won(state))):
        print("{}{}\t| {}\t\t\t| {:1.4f}\t| {:1.4f}\t\t| {} ({:3.2f}%){}".format("\033[1m\033[92m" if has_won(state) else "", episode + 1, step - 1, epsilon, total_training_reward / (episode + 1), successful_training_episodes, successful_training_episodes / (episode + 1) * 100, ", SOLVED!!!\033[0m" if has_won(state) else ""))
    
    if (verbosity > 1):
        print("Q-Table:")
        print(qtable)
        print("Board on GameOver:")
        env.render()
    
    epsilon = np.clip((1 - decay_rate) * epsilon, min_epsilon, max_epsilon)

Episode	| Steps survived	| Epsilon 	| Average Reward	| Successful Episodes
10000	| 22			| 0.9048	| 0.0045		| 45 (0.45%)
20000	| 17			| 0.8187	| 0.0057		| 114 (0.57%)
30000	| 29			| 0.7408	| 0.0079		| 237 (0.79%)
40000	| 5			| 0.6703	| 0.0116		| 465 (1.16%)
50000	| 12			| 0.6065	| 0.0171		| 853 (1.71%)
60000	| 16			| 0.5488	| 0.0254		| 1526 (2.54%)
70000	| 15			| 0.4966	| 0.0362		| 2536 (3.62%), SOLVED!!!
80000	| 19			| 0.4493	| 0.0497		| 3979 (4.97%)
90000	| 19			| 0.4066	| 0.0653		| 5875 (6.53%)
100000	| 10			| 0.3679	| 0.0833		| 8330 (8.33%)


# Final Q-Table

In [7]:
print("Successful Episodes: {}, Success Rate: {}".format(successful_training_episodes, successful_training_episodes / total_training_episodes))
print(qtable)

Successful Episodes: 8330, Success Rate: 0.0833
[[0.48767498 0.51334208 0.51334208 0.48767498]
 [0.48767498 0.54036009 0.54036009 0.51334208]
 [0.51334208 0.56880009 0.56880009 0.54036009]
 [0.54036009 0.59873694 0.59873694 0.56880009]
 [0.56880009 0.63024941 0.63024941 0.59873694]
 [0.59873694 0.66342043 0.66342043 0.63024941]
 [0.63024941 0.6983373  0.6983373  0.66342043]
 [0.66342043 0.73509189 0.6983373  0.6983373 ]
 [0.51334208 0.54036009 0.54036009 0.48767498]
 [0.51334208 0.56880009 0.56880009 0.51334208]
 [0.54036009 0.59873694 0.59873694 0.54036009]
 [0.56880009 0.         0.63024941 0.56880009]
 [0.59873694 0.66342043 0.66342043 0.59873694]
 [0.63024941 0.6983373  0.6983373  0.63024941]
 [0.66342043 0.73509189 0.73509189 0.66342043]
 [0.6983373  0.77378094 0.73509189 0.6983373 ]
 [0.54036009 0.56880009 0.56880009 0.51334208]
 [0.54036009 0.59873694 0.59873694 0.54036009]
 [0.56880009 0.63024941 0.         0.56880009]
 [0.         0.         0.         0.        ]
 [0.        

# Let the Agent Play the Game with our Q-Table now

In [8]:
successful_episodes = 0
consecutive_successes = 0
consecutive_successes_record = 0

for episode in range(total_play_episodes):
    state = env.reset()
    done = False
    step = 0
    
    while (not done):
        action = np.argmax(qtable[state])
        state, reward, done, info = env.step(action)
        step += 1
    
    if (verbosity > 0):
        print("\n{}Episode finished after {} steps. {}".format("\033[1m\033[92m" if has_won(state) else "", step, "SOLVED!!!\033[0m" if has_won(state) else ""))
        env.render()
    
    if (has_won(state)):
        successful_episodes += 1
        consecutive_successes += 1
        
        if (consecutive_successes > consecutive_successes_record):
            consecutive_successes_record = consecutive_successes
    else:
        consecutive_successes = 0

print("Successful Episodes: {}, Success Rate: {:3.2f}%, Most Consecutive Successes: {}".format(successful_episodes, successful_episodes / total_play_episodes * 100, consecutive_successes_record))

Successful Episodes: 10000, Success Rate: 100.00%, Most Consecutive Successes: 10000


# Successfull Q-Tables:

In [9]:
#[[5.95622600e-03 7.69065614e-03 1.29795247e-02 7.66528198e-03]
# [8.74553016e-03 8.65319714e-03 8.72166674e-03 1.37515984e-02]
# [8.81988918e-03 8.74345842e-03 1.94510759e-02 9.85717695e-03]
# [1.18944941e-02 2.06338083e-02 1.22873574e-02 1.19921980e-02]
# [1.73632911e-02 1.93390616e-02 2.04212675e-02 2.57357832e-02]
# [2.41697915e-02 2.13073046e-02 2.40035491e-02 3.65479906e-02]
# [3.16741568e-02 3.78067173e-02 3.76115616e-02 4.02400611e-02]
# [4.24389580e-02 4.10045871e-02 4.15269458e-02 4.12636846e-02]
# [7.59993744e-03 5.07833173e-03 1.22899596e-02 7.67360036e-03]
# [8.34331784e-03 8.76222400e-03 5.39142687e-03 1.26694183e-02]
# [6.01496547e-03 7.06392586e-03 7.29510495e-03 1.47896524e-02]
# [2.92117077e-03 1.38889310e-04 2.93924912e-03 1.90603093e-02]
# [1.39507785e-02 9.62424956e-03 1.37226170e-02 1.96127576e-02]
# [2.44461972e-02 1.23096432e-02 2.08778632e-02 3.22170692e-02]
# [3.70801023e-02 4.18644622e-02 3.95616535e-02 4.24286478e-02]
# [4.35499884e-02 4.23297843e-02 4.46662107e-02 4.28473276e-02]
# [4.85419705e-03 3.57217550e-03 4.85763621e-03 9.74635854e-03]
# [1.98756958e-03 1.66430775e-03 6.62345637e-03 7.28144773e-03]
# [6.44407402e-03 1.03278578e-03 7.52992239e-04 6.25531512e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [9.86193984e-04 2.00869677e-03 2.81902240e-02 2.35364932e-05]
# [7.02054894e-03 1.21763063e-03 1.17078996e-02 4.50884608e-02]
# [4.43281104e-02 3.67499740e-02 1.01661659e-01 4.72615748e-02]
# [4.60719795e-02 2.64849312e-01 5.62544578e-02 4.97058468e-02]
# [2.79362830e-03 1.98764438e-03 2.43891370e-03 1.15725606e-02]
# [3.04840849e-03 2.90759350e-03 4.83321013e-03 5.43260744e-04]
# [2.80367515e-04 4.67973083e-04 1.75630797e-04 6.35989310e-03]
# [3.52951718e-10 8.83436848e-08 9.47171686e-08 1.16648248e-03]
# [7.92521972e-03 1.04370603e-04 4.51747821e-05 2.42738860e-05]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.96143817e-02 2.89523994e-02 2.38031086e-01 1.56725925e-02]
# [5.49468161e-02 6.16086796e-02 1.27851490e-01 6.29401471e-02]
# [4.69556282e-04 2.56309244e-03 5.23205406e-04 8.58511128e-03]
# [5.93768821e-05 5.37151889e-05 3.12819175e-04 8.00110272e-03]
# [2.11282516e-07 1.07990752e-09 1.76098119e-12 3.01649255e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.60768992e-04 8.78985324e-05 1.52134288e-02 1.93373189e-04]
# [4.59251125e-05 3.95078011e-02 1.75938417e-04 1.57314475e-03]
# [1.78047173e-03 9.98981879e-04 8.90742621e-03 1.41476471e-01]
# [1.32630260e-01 6.90415944e-02 3.82889137e-01 1.22856861e-01]
# [2.85430769e-03 3.65072096e-05 8.80596645e-05 2.29044875e-05]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.76082381e-08 1.66950115e-07 7.05655667e-06 1.05006010e-09]
# [1.65966365e-06 9.78126002e-09 2.45898537e-05 1.88988008e-03]
# [2.44260590e-02 3.13394824e-03 8.08105403e-05 2.84770156e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.39932865e-01 6.16363350e-02 7.71049639e-01 6.11065910e-02]
# [1.53596945e-03 1.50959083e-06 2.17785434e-05 4.24965531e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [8.06125006e-09 1.25214214e-08 3.08602142e-04 2.81551190e-09]
# [5.74118960e-05 1.27359187e-10 1.18211467e-10 2.50289029e-10]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.53403447e-04 3.87738188e-06 2.74009624e-02 1.37660814e-13]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [4.22635205e-03 1.65629656e-01 9.96055200e-01 2.53205922e-02]
# [1.34464759e-03 1.25840735e-03 6.75619428e-05 9.90625422e-04]
# [2.11688699e-07 1.40684221e-03 4.71952062e-06 1.43773972e-06]
# [1.21293041e-03 1.03651765e-06 5.10582741e-07 8.32671350e-07]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.43436982e-05 2.71140378e-04 1.98503963e-03 7.10405904e-06]
# [1.94681063e-01 5.16025328e-02 4.08468117e-01 1.01699777e-01]
# [2.52934638e-02 5.67599658e-01 1.24092961e-01 1.16560682e-01]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

# 50000 Total Training Episodes
# Successful Episodes: 173519, Success Rate: 0.347038
#[[2.19549647e-02 2.17581636e-02 2.30287258e-02 2.24937164e-02]
# [2.45807559e-02 5.52544062e-02 2.46254976e-02 2.46329635e-02]
# [1.80817235e-02 5.25186945e-03 6.16385170e-02 3.09600180e-02]
# [2.13607426e-02 2.17131811e-02 5.99103692e-02 5.90855967e-03]
# [2.78457274e-02 2.61901375e-02 9.28584504e-02 2.39914521e-02]
# [1.29879381e-02 3.21750413e-02 8.71393917e-02 4.48727752e-02]
# [3.35216651e-02 6.78184242e-02 7.89616136e-02 3.35767698e-02]
# [9.26536876e-02 7.78656889e-02 6.35847062e-02 1.79737562e-02]
# [2.02929275e-02 6.45045557e-03 1.06724383e-02 1.93715831e-02]
# [4.12128313e-03 7.38666325e-03 4.08388576e-03 4.43733583e-02]
# [3.60697787e-03 1.29205325e-02 1.04713164e-02 4.73359132e-02]
# [1.73564803e-03 2.51783576e-02 1.41936617e-02 9.96376526e-02]
# [6.58863973e-03 7.25515823e-03 2.20116852e-02 1.07147163e-01]
# [4.21295987e-02 1.59375440e-02 1.09785715e-01 1.57313863e-02]
# [3.81608124e-02 1.00239886e-01 4.16836636e-02 4.15745474e-02]
# [1.81237559e-01 6.80455036e-02 7.04207323e-02 4.27137727e-02]
# [3.75462983e-03 2.19065396e-02 2.95004585e-03 2.04240932e-02]
# [2.35770787e-03 1.91366962e-03 2.74283162e-03 2.68805321e-02]
# [5.71696321e-03 1.05241601e-04 1.98866626e-03 6.93969544e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.45703332e-05 4.66474400e-04 4.34956941e-02 1.82898129e-03]
# [3.07973626e-05 1.20112964e-03 6.72970133e-03 1.29056265e-01]
# [9.34464502e-02 6.22878497e-02 2.11668769e-01 7.68678885e-02]
# [6.06518658e-02 2.08638729e-01 5.90221476e-02 5.70317965e-02]
# [2.99868394e-03 2.58302763e-03 2.44191313e-03 2.80395240e-03]
# [1.59300161e-03 8.10871884e-04 5.90466341e-03 1.14998334e-03]
# [9.27346078e-04 6.91661615e-05 1.29527120e-04 1.90656940e-04]
# [1.71565364e-06 5.76475858e-03 1.04041533e-08 5.94300973e-06]
# [4.27410206e-03 6.29733777e-05 6.58665242e-04 4.10884695e-03]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [8.17302026e-04 9.60726641e-05 1.90097543e-01 1.09390467e-03]
# [1.98417021e-01 1.84594476e-01 9.04918861e-02 6.71138740e-02]
# [8.85498599e-04 7.34252140e-04 1.01656590e-03 2.50798192e-03]
# [6.50401657e-04 4.32939838e-05 2.10139070e-05 8.66158021e-04]
# [7.66126822e-08 1.87352427e-06 1.34301352e-07 1.81904612e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.93634496e-05 2.08980141e-04 1.31998172e-01 3.88512261e-04]
# [1.18037950e-02 1.13751715e-01 7.46358629e-03 8.18847292e-05]
# [2.03424139e-03 6.20122733e-02 8.28225142e-04 2.18931287e-01]
# [7.90015398e-02 2.54996415e-01 1.96793859e-01 1.98281355e-01]
# [1.18657545e-03 1.88997003e-04 1.19223184e-04 2.23411630e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.26554297e-14 5.14093791e-08 2.22594392e-05 2.11623546e-07]
# [4.99648544e-07 4.82589823e-06 1.07479816e-04 5.84476291e-02]
# [8.51017601e-02 1.33732556e-05 3.32945081e-05 1.65913528e-03]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.30441949e-03 3.48077319e-02 3.03640635e-01 3.44889420e-03]
# [8.86931659e-04 1.65273540e-06 2.24100267e-04 3.41071476e-06]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.01902370e-08 2.88029741e-09 1.62752696e-06 5.66020061e-07]
# [3.05971570e-04 2.47121172e-08 2.59046765e-10 2.87107183e-06]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.46571420e-06 1.45634456e-08 8.96206456e-02 1.52447329e-06]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.67568638e-01 1.96747464e-01 7.85298787e-01 1.54589243e-01]
# [6.41463341e-04 8.37438530e-05 1.46721512e-04 1.32503269e-04]
# [1.59225807e-05 7.20568974e-04 1.06490465e-05 5.79235727e-08]
# [5.71985873e-05 3.03106331e-09 3.17554261e-07 1.46262145e-07]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.32610951e-05 1.82403373e-05 2.90993088e-05 5.95929718e-02]
# [3.37421524e-01 1.24584589e-01 4.53448525e-01 7.84905081e-02]
# [1.56422702e-01 9.59663405e-01 3.71583647e-02 3.34055793e-02]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

# 100000 Total Training Episode
# Successful Episodes: 6919, Success Rate: 69.19%, Most Consecutive Successes: 23
#[[2.16514527e-02 2.10875844e-02 2.13169850e-02 2.16577894e-02]
# [2.20817289e-02 2.32282477e-02 2.16250815e-02 2.22314616e-02]
# [2.52593968e-02 3.85689415e-02 1.77165270e-02 1.94583384e-02]
# [2.44333401e-02 3.70271186e-02 1.23232721e-01 3.81525640e-02]
# [5.86884285e-02 8.70811015e-02 3.08096750e-02 5.13936902e-02]
# [4.06913257e-02 6.72312591e-02 7.53706665e-02 3.71846449e-02]
# [4.76084244e-02 7.00594635e-02 1.47721733e-01 7.34143329e-02]
# [4.50868007e-02 1.54349260e-01 4.52779931e-02 4.61678837e-02]
# [1.11603922e-02 2.12476206e-02 3.17160240e-02 2.06367974e-02]
# [7.95423277e-03 1.21946472e-02 1.93459442e-02 3.28289847e-02]
# [2.31767027e-02 1.85429340e-02 1.30846839e-02 7.21144182e-02]
# [9.12942791e-03 9.27173791e-04 1.21209609e-02 4.60427854e-02]
# [2.37250912e-02 1.11039759e-01 4.24368007e-02 3.91433828e-02]
# [5.95288303e-02 3.40304780e-02 1.21037760e-01 3.90410030e-02]
# [7.36559234e-02 5.04757278e-02 2.05545956e-01 4.78037457e-02]
# [8.01912237e-02 2.06265652e-01 7.90359998e-02 8.01778262e-02]
# [1.99588566e-02 5.39198990e-03 7.32372239e-03 5.87906090e-03]
# [1.18807373e-02 2.00530588e-02 2.16838408e-03 3.08528559e-03]
# [1.53343856e-02 2.04511922e-04 4.30763008e-04 2.41261060e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [1.40817377e-03 1.91024061e-03 3.87986876e-02 1.59007029e-04]
# [8.63997336e-03 3.56692073e-03 2.11188234e-03 1.13770039e-01]
# [9.12439942e-02 5.14524232e-02 3.77400889e-01 6.58370324e-02]
# [1.13689489e-01 1.15162827e-01 5.18059128e-01 1.17836964e-01]
# [3.58636288e-03 5.07543934e-03 1.38738049e-02 1.58466453e-02]
# [2.39982986e-03 1.49230449e-03 2.09156682e-03 1.83800511e-02]
# [1.47766523e-03 1.04852240e-03 1.27693683e-03 2.35375689e-02]
# [1.85601859e-07 1.16799949e-04 1.93614377e-07 1.68239280e-02]
# [1.32485822e-02 2.39187064e-04 7.49403323e-04 7.34502241e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [9.50076829e-03 1.11154261e-02 5.44375387e-01 3.11039707e-02]
# [1.18055982e-01 1.01867691e-01 5.82734971e-01 1.14326443e-01]
# [7.55274224e-04 1.06984935e-03 8.24578487e-04 8.92940802e-03]
# [1.49467055e-03 5.55542308e-05 5.49861332e-05 1.92139172e-04]
# [2.29419677e-03 4.02732867e-08 4.69584747e-07 8.82528732e-07]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [2.98557405e-04 4.80937025e-05 1.60166884e-02 6.19920165e-04]
# [1.56947797e-02 4.71873848e-02 2.44169667e-03 1.79583710e-03]
# [6.64854589e-03 1.20302199e-01 1.58916510e-02 4.70115652e-02]
# [1.44871783e-01 1.42891853e-01 7.52988182e-01 4.17245892e-01]
# [6.16879490e-03 4.36319430e-04 3.09666997e-04 2.74290519e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [7.20015302e-12 6.85208598e-05 2.56746048e-09 7.73919469e-09]
# [7.27624190e-04 2.84555792e-06 7.27306160e-06 3.26558713e-03]
# [1.29021233e-02 8.68125114e-05 6.44964965e-04 4.31058915e-05]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [5.39015771e-04 1.06141145e-01 9.18843174e-01 1.94756467e-01]
# [5.98648070e-03 3.04628242e-04 2.61346076e-04 1.55763917e-04]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [9.24693216e-06 1.16936010e-09 4.51591941e-05 9.62499470e-09]
# [4.51188764e-05 1.05189663e-11 1.08314200e-08 1.45267076e-10]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [5.49128201e-02 8.74009346e-04 4.47596456e-04 1.90435848e-08]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [3.30132348e-02 1.35895752e-01 9.90082773e-01 3.93256284e-03]
# [1.52346406e-03 2.14622062e-04 1.60396657e-04 2.20632394e-04]
# [5.71839443e-05 5.13883987e-04 1.79573302e-06 5.53753369e-06]
# [3.96683121e-04 3.81855511e-07 6.17279773e-09 2.89119553e-05]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
# [9.21312063e-09 1.79918002e-08 6.66069039e-08 4.37477871e-08]
# [4.31420028e-02 2.77544959e-02 3.65833191e-01 2.28298387e-02]
# [9.24466081e-02 7.85204505e-01 3.35487798e-02 1.28052136e-01]
# [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]